## Import

In [9]:
import pandas as pd
import numpy as np

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

KeyboardInterrupt: 

## Data Load

In [ ]:
# 이미 pca 처리된 값으로 예상, 2013 캐글 이상치 탐지 데이터와 비슷하다

In [ ]:
train_df = pd.read_csv('../open/train.csv') # Train
train_df.head()

In [ ]:
len(train_df)

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
X  = train_df.iloc[:,1:].values

In [ ]:
pca = PCA(n_components=2,)
pca.fit(X)

In [ ]:
print(pca.transform(X))
pca_X = pca.transform(X)

In [ ]:
df_pca_0 = pd.DataFrame(pca_X,columns=['component 0','component 1'])

In [ ]:
print(pca.explained_variance_ratio_[0])

In [ ]:
plt.plot(pca_X)

In [ ]:
plt.scatter(df_pca_0['component 0'], df_pca_0['component 1'], color = 'orange', alpha = 0.7, label = 'setosa')

In [ ]:
val_df = pd.read_csv('../open/val.csv') # Validation
val_df.head()

In [ ]:
val_df.Class.value_counts()

In [ ]:
#

In [ ]:
#이상치값
val_X_1  = pca.transform(val_df[val_df.Class ==1].iloc[:,1:-1])

In [ ]:
val_X_0  = pca.transform(val_df[val_df.Class ==0].iloc[:,1:-1])

In [ ]:
df_pca_val_1 = pd.DataFrame(val_X_1, columns=['component 0','component 1'])
## Train/Validation Feature 분포 확인

In [ ]:
df_pca_val_0 = pd.DataFrame(val_X_0, columns=['component 0','component 1'])

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.scatter(df_pca_val_0['component 0'], df_pca_val_0['component 1'], color = 'orange', alpha = 0.7, label = 'setosa')
plt.xlim(-2,8)
plt.ylim(-2,25)
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.scatter(df_pca_val_1['component 0'], df_pca_val_1['component 1'], color = 'red', alpha = 0.7, label = 'setosa')
plt.xlim(-2,8)
plt.ylim(-2,25)
plt.show()

In [ ]:
# 좀 더 구분되는 특징은?
# -2~6 과 0~ 25 사이의 range만 가지는 이상치 값, 이 range에서 정상 데이터의 분포는?
plt.scatter(df_pca_0['component 0'], df_pca_0['component 1'], color = 'orange', alpha = 0.7, label = 'setosa')

In [ ]:
train_df.drop(columns=['ID']).hist(bins = 50, figsize = (20,20))
plt.show()

In [ ]:
val_df.drop(columns=['ID', 'Class']).hist(bins = 50, figsize = (20,20))
plt.show()

In [ ]:
## Validation set 사기 거래 비율
#### (*) Validation set의 사기 거래 비율이 다른 데이터집합에서도 비슷하게 발생할 것이라고 가정

In [ ]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

In [ ]:
## Model Define & Fit

In [ ]:
# Train dataset은 Label이 존재하지 않음
train_x = train_df.drop(columns=['ID']) # Input Data

In [ ]:
# 가설 설정 : Train dataset도 Validation dataset과 동일한 비율로 사기거래가 발생 했을 것이다. -> model parameter : contamination=val_contamination(=0.001055) 적용
model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x)

In [ ]:
## Evaluation : Validation set

In [ ]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [ ]:
val_x = val_df.drop(columns=['ID', 'Class']) # Input Data
val_y = val_df['Class'] # Label

val_pred = model.predict(val_x) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

In [ ]:
## Inference : Test set

In [ ]:
test_df = pd.read_csv('../open/test.csv') # Train
test_df.head()

In [ ]:
test_x = test_df.drop(columns=['ID'])

In [ ]:
test_pred = model.predict(test_x) # model prediction
test_pred = get_pred_label(test_pred)

In [ ]:
## Submission

In [ ]:
submit = pd.read_csv('../open/sample_submission.csv')
submit.head()

In [ ]:
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)

In [ ]:
v_feature = [ c for c  in train_df.columns if c.find('V') != -1]


In [ ]:
import lightgbm as lgb

In [ ]:
train_df

In [ ]:
lgb.Dataset(t)

In [ ]:
fraud = val_df[val_df.Class ==1].iloc[:,1:]
clean = val_df[val_df.Class ==0].iloc[:,1:]

In [ ]:
# undersample clean transcations
RATIO_TO_FRAUD = 15
RANDOM_SEED = 42

In [ ]:
clean_undersampled = clean.sample(
    int(len(fraud) * RATIO_TO_FRAUD)
    ,random_state=RANDOM_SEED
)

In [ ]:
clean_undersampled

In [ ]:
clean_undersampled

In [ ]:
# concatenate with fraud transactions into a single dataframe
visualization_initial = pd.concat([fraud,clean_undersampled])

In [ ]:
column_names = list(visualization_initial.drop('Class',axis=1).columns)

In [ ]:
feature , labels = visualization_initial.drop('Class',axis=1).values , visualization_initial.Class.values

In [ ]:
feature


In [ ]:
labels


In [ ]:
print(f"""The non-fraud dataset has been undersampled from {len(clean):,} to {len(clean_undersampled):,}.
      This represents a ratio of {RATIO_TO_FRAUD}:1 to fraud.""")

In [ ]:
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D

def tsne_scatter(features , labels , dimensions=2, save_as='graph.png'):
    if dimensions not in (2,3):
        raise ValueError('tsne_scatter can only plot in 2d or 3d (What are you ? An alien that can visualize >3d?).Make sure the "dimensions" argument is in (2,3)')
    # t-SNE dimensionality reduction
    features_embedded = TSNE(n_components=dimensions, random_state=RANDOM_SEED).fit_transform(features)


    # initializing the plot
    fig , ax = plt.subplots(figsize=(8,8))
    # counting dimensions
    if dimensions ==3: ax = fig.add_subplot(111, projection='3d')

    # plotting data
    ax.scatter(
        *zip(*features_embedded[np.where(labels==1)]),
        marker='o',
        color='r',
        s=2,
        alpha=0.7,
        label='Fraud'

    )
    ax.scatter(
        *zip(*features_embedded[np.where(labels==0)]),
        marker='o',
        color='g',
        s=2,
        alpha=0.3,
        label='Clean'
    )

    # storing it to be displayed later
    plt.legend(loc='best')
    plt.savefig(save_as);
    plt.show()

In [ ]:
tsne_scatter(feature,labels,dimensions=2,save_as='tsne_initial_2d.png')
# some clusters are apparent , but a minority of fraud transcations remains sneaky , sneaky

In [ ]:

# Training : only non-fraud
# 섞여 있어서 알 수 없다 이 데이터는
# 많은 batch size로 적게 영향을 주는 방법이 유효할 듯

In [ ]:
v_feature = [ c for c  in train_df ]

In [ ]:
val_df = pd.read_csv('../open/val.csv') # Validation
val_df.head()

## Train/Validation Feature 분포 확인

In [ ]:
train_df.drop(columns=['ID']).hist(bins = 50, figsize = (20,20))
plt.show()

In [ ]:
val_df.drop(columns=['ID', 'Class']).hist(bins = 50, figsize = (20,20))
plt.show()

## Validation set 사기 거래 비율
#### (*) Validation set의 사기 거래 비율이 다른 데이터집합에서도 비슷하게 발생할 것이라고 가정

In [ ]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

## Model Define & Fit

In [ ]:
# Train dataset은 Label이 존재하지 않음
train_x = train_df.drop(columns=['ID']) # Input Data

In [ ]:
# 가설 설정 : Train dataset도 Validation dataset과 동일한 비율로 사기거래가 발생 했을 것이다. -> model parameter : contamination=val_contamination(=0.001055) 적용
model = IsolationForest(n_estimators=125, max_samples=len(train_x), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x)

## Evaluation : Validation set

In [ ]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [ ]:
val_x = val_df.drop(columns=['ID', 'Class']) # Input Data
val_y = val_df['Class'] # Label

val_pred = model.predict(val_x) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

## Inference : Test set

In [ ]:
test_df = pd.read_csv('../open/test.csv') # Train
test_df.head()

In [ ]:
test_x = test_df.drop(columns=['ID'])

In [ ]:
test_pred = model.predict(test_x) # model prediction
test_pred = get_pred_label(test_pred)

## Submission

In [ ]:
submit = pd.read_csv('../open/sample_submission.csv')
submit.head()

In [ ]:
submit['Class'] = test_pred
submit.to_csv('./submit.csv', index=False)

In [ ]:
a = "123456"

In [ ]:
print(a[7:0:-1])